# Python 1 Workshop

Now that most of you have learned the basics of Python (variables, types, strings, data structures, loops, and functions) from our Python 0 workshop, we are ready to dive in to some more advanced Python topics. In this workshop we will learn how to use Python for data science, and introduce you to the basics of data science in Python. 

# Data Science Tutorial

Now that we've covered some Python basics, we will begin a tutorial going through many tasks a data scientist may perform.  We will obtain real world data and go through the process of auditing, analyzing, visualing, and building classifiers from the data.

We will use a database of selected disease statistics of various contries from the Global Health Observatory. The data is organized by country and year, with the number of specific incidents of each disease listed. The attributes and domain of each entry are described by the table below:

| Attribute                     | Domain                          |
|-------------------------------|---------------------------------|
| 1. Country                    | String                          |
| 2. Year                       | Year (2009-2014)                |
| 3. T.b. gambiense             | Integer                         |
| 4. T.b. rhodesiense           | Integer                         |
| 5. Cholera                    | Integer                         |
| 6. Meningitis (suspected)     | Integer                         |
| 7. Congenital Rubella         | Integer                         |
| 8. Diphtheria                 | Integer                         |
| 9. Japanese encephalitis      | Integer                         |
| 10. Leprosy                   | Integer                         |
| 11. Malaria                   | Integer                         |
| 12. Measles                   | Integer                         |
| 13. Mumps                     | Integer                         |
| 14. Neonatal Tetanus          | Integer                         |
| 15. Pertussis                 | Integer                         |
| 16. Plague                    | Integer                         |
| 17. Poliomyelitis             | Integer                         |
| 18. Rubella                   | Integer                         |
| 19. Total Tetanus             | Integer                         |
| 20. Tuberculosis              | Integer                         |
| 21. Yellow Fever              | Integer                         |
| 22. Cutaneous Leishmaniasis   | Integer                         |
| 23. Visceral Leishmaniasis    | Integer                         |

For more information on this data set:
http://apps.who.int/gho/data/node.home

## Obtaining the Data
Lets begin by programmatically obtaining the data.  Here I'll define a function we can use to make HTTP requests and download the data

In [1]:
#define download_file function
def download_file(url, local_filename):
    import requests
    
    # stream = True allows downloading of large files; prevents loading entire file into memory
    r = requests.get(url, stream = True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                f.flush()

Now we'll specify the url of the file and the file name we will save to

In [9]:
#specify url and filename


And make a call to <code>download_file</code>

In [8]:
#call our download_file function


**Note:**  If you see an InsecurePlatformWarning message, ignore it. More info can be found here: https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning

Now this might seem like overkill for downloading a single, small csv file, but we can use this same function to access countless APIs available on the World Wide Web by building an API request in the url.

## Wrangling the Data
Now that we have some data, lets get it into a useful form.  For this task we will use a package called pandas. pandas is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for Python.  The most fundamental data structure in pandas is the dataframe, which is similar to the data.frame data structure found in the R statistical programming language.

For more information: http://pandas.pydata.org

pandas dataframes are a 2-dimensional labeled data structures with columns of potentially different types.  Dataframes can be thought of as similar to a spreadsheet or SQL table.

There are numerous ways to build a dataframe with pandas.  Since we have already attained a csv file, we can use a parser built into pandas called <code>read_csv</code> which will read the contents of a csv file directly into a data frame.

For more information: http://pandas.pydata.org/pandas-docs/dev/generated/pandas.io.parsers.read_csv.html

In [7]:
#import pandas and load dataset into a frame
#import the module and alias it as pd

#show the first few rows of the data


Lets take a look at some simple statistics for the **Cholera** column

In [4]:
#describe cholera column


Referring to the documentation, the data contains 1164 entries. However, if we take a look at the "count" section, it shows only 245 entries. This is because the original data is filled with empty strings, which pandas automatically converts to Numpy's <code>nan</code> datatype, or "Not a Number". 

Lets take a look at another column, this time **Pertussis**

In [3]:
#describe pertussis column


Well at least the name is correct.  We were expecting a mean and standard deviation, and now the data type is an object.  

Whats up with our data?

We have arrived at arguably the most important part of performing data science: dealing with messy data.  One of most important tools in a data scientist's toolbox is the ability to audit, clean, and reshape data.  The real world is full of messy data and your sources may not always have data in the exact format you desire.

In this case we are working with csv data, which is a relatively straightforward format, but this will not always be the case when performing real world data science.  Data comes in all varieties from csv all the way to something as unstructured as a collection of emails or documents.  A data scientist must be versed in a wide variety of technologies and methodologies in order to be successful.

Now, lets do a little bit of digging into why were are not getting a numeric pandas column

In [2]:
#find unique values


Using <code>unique</code> we can see that '0 0', '5 5', and '2 2' all appear as distinct values in this series. Because of the space between the numbers, Python has classified these as *strings* rather than *integers*. Indeed, it's not immediately obvious that these were meant to be legitimate entries in the first place.

Lets see what we can do with these unrecognized values. 

In [1]:
#convert column to numeric values


Here we have attempted to convert the **Pertussis** series to a numeric type.  Lets see what the unique values are now.

In [66]:
#find new unique values

The decimal point after each number means that it is an integer value being represented by a floating point number.  Now instead of our pesky *strings* we have <code>nan</code> (not a number).  <code>nan</code> is a construct used by pandas to represent the absence of value.  It is a data type that comes from the package numpy, used internally by pandas, and is not part of the standard Python library.

Now that we have <code>nan</code> values in place of strings, we can use some nice features in pandas to deal with these missing values.

What we are about to do is what is called "imputing" or providing a replacement for missing values so the data set becomes easier to work with.  There are a number of strategies for imputing missing values, all with their own pitfalls.  In general, imputation introduces some degree of bias to the data, so the imputation strategy taken should be in an attempt to minimize that bias.

Here, we will simply ignore all of the <code>nan</code> values, however other strategies such as replacing the <code>nan</code>'s with the mean of the data are also commonly used.

In [67]:
#convert whole data frame

<code>health_data.mean().round()</code> will take the mean of each column (this computation ignores the currently present nan values), then round, and return a dataframe indexed by the columns of the original dataframe.

This function can be used to replace all missing values with the mean of each column. In this tutorial however, we will not use this method, because the large number of missing values would greatly skew our standard deviations.

In [68]:
#find mean values for imputing

Now that we have figured out how to impute these missing values, lets start over and quickly apply this technique to the entire dataframe.

In [69]:
#quickly load and convert data

In [70]:
#check unique values

Structurally, Pandas dataframes are a collection of Series objects sharing a common index.  In general, the Series object and Dataframe object share a large number of functions with some behavioral differences.  In other words, whatever computation you can do on a single column can generally be applied to the entire dataframe.

Now we can use the dataframe version of <code>describe</code> to get an overview of all of our data

In [71]:
#overview description of data frame

## Visualizing the Data
Another important tool in the data scientist's toolbox is the ability to create visualizations from data.  Visualizing data is often the most logical place to start getting a deeper intuition of the data.  This intuition will shape and drive your analysis.

Even more important than visualizing data for your own personal benefit, it is often the job of the data scientist to use the data to tell a story.  Creating illustrative visuals that succinctly convey an idea are the best way to tell that story, especially to stakeholders with less technical skillsets.

Here we will be using a Python package called ggplot (https://ggplot.yhathq.com).  The ggplot package is an attempt to bring visuals following the guidelines outlayed in the grammar of graphics (http://vita.had.co.nz/papers/layered-grammar.html) to Python.  It is based off of and intended to mimic the features of the ggplot2 library found in R.  Additionally, ggplot is designed to work with Pandas dataframes, making things nice and simple. 

We'll start by doing a bit of setup

If you are using a Mac, try installing ggplot by opening the terminal and running the command "pip install ggplot"
For Windows users, try opening the anaconda terminal and running the same command, or the command "conda install ggplot"

In [72]:
# The following line is NOT Python code, but a special syntax for enabling inline plotting in IPython
%matplotlib inline 

from ggplot import *

import warnings

# ggplot usage of pandas throws a future warning
warnings.filterwarnings('ignore')

ImportError: No module named ggplot

So we enabled plotting in IPython and imported everything from the ggplot package.  Now we'll create a plot and then break down the components

In [ ]:
#create our first plot

A plot begins with the <code>ggplot</code> function.  Here, we pass in the cancer_data pandas dataframe and a special function called <code>aes</code> (short for aesthetic).  The values provided to <code>aes</code> change depending on which type of plot is being used.  Here we are going to make a histogram from the **human African trypanosomiasis (T.b. rhodesiense)** column in health_data, so that column name needs to be passed as the x parameter to <code>aes</code>.

The grammar of graphics is based off of a concept of "geoms" (short for geometric objects).  These geoms provide granular control of the plot and are progressively added to the base call to <code>ggplot</code> with + syntax.


Lets say we wanted to show the mean number of cases on this plot.  We could do something like the following

In [ ]:
#plot with vline

As you can see, each geom has its own set of parameters specific to the appearance of that geom (also called aesthetics).

Lets try a scatter plot to get some multi-variable action

In [ ]:
#scatter plot

With a simple aesthetic addition, we can see how these two variables have changed over the past six years.

In [ ]:
#color scatter plot

Similar to the vline geom we added to our histogram, we can also add an abline geom. The vline and hline geoms are fixed in some way (plot vertical lines with vline and horizontal lines with hline) but the abline geom can be used to plot a line of any slope or intercept.

Here, we are going to use it to see how our scatter plot compares to the line y=x. It's not exactly too useful for our plot, but it can be useful for other plots you decide to make, such as plotting a best fit line against a linear plot!

In [2]:
#scatter plot with abline

By adding <code>color = 'Year'</code> as a parameter to the aes function, we now give a color to each unique value found in that column and automatically get a legend.

We can also do things such as add a title or change the axis labeling with geoms

To see more ways you can edit your plots, check out this link: http://yhat.github.io/ggpy/

In [ ]:
#plot with custom title and axes

I highly encourage you to check out https://ggplot.yhathq.com/docs/index.html to see all of the available geoms.  The best way to learn is to play with and visualize the data with many different plots and aesthetics.

There doesn't seem to be much correlation between these two variables as a function of the year, however it is difficult to say with such simple statistics. If we wanted to analyze relationships between all of the variables in our data set, we would need to analyze a very high-dimensional space. Using some more complex data analysis and machine learning techniques, we may be able to extract higher order correlations in this high-dimensional data space. This will be the topic of our next workshop, Python II.

## Summary and Python II

In the second part of our three-part Python series, we've learned about variables, data structures, functions, and graphing. While we have introduced these topics in the context of data science with Python, they are central to programming in any language and in any context. We have also laid the foundation for programmatically obtaining, cleaning, and visualizing data sets.

Now that we have an understanding of how to obtain and visualize some simple statistical information contained in a dataset, we've set the stage for machine learning and data analysis. These topics will be covered in depth in our next workshop, Python II.

Please fill out the post workshop survey posted on the Facebook event page!

Don't forget about our next workshop, Python 2, and our upcoming workshops this semester, including the R workshop series, sports data analysis in R, Data Visualization in Python, and more!